In [2]:
import pandas as pd
from io import StringIO 
from geopandas import GeoDataFrame
from iclientpy.rest.api.model import DataItemType,PermissionType,DataPermissionType
from iclientpy.portal import BaseLayerType,MapShareSettingBuilder,DataShareSettingBuilder
from iclientpy.online import Online

/opt/conda/lib/python3.6/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/opt/conda/lib/python3.6/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


ImportError: cannot import name 'DataItemType'

填写自己的Online用户名和密码，然后删除#以取消注释代码

In [ ]:
#username = '你的用户名'
#password = '你的密码'
online=Online(username, password)

从CSV文件读取数据

In [ ]:
df=pd.read_csv("province.csv")
df["2015年"]=pd.to_numeric(df["2015年"],errors='coerce')

获取地理位置数据，geonjson格式

In [ ]:
gdf=GeoDataFrame.from_file('./data.json')

将CSV数据与地理位置数据进行拼接，根据省份名称所在字段名进行数据拼接

In [ ]:
result=gdf.merge(df,left_on='name',right_on='地区')
result.drop(['name'],axis=1,inplace=True)

上传数据到Online

In [ ]:
data_id = online.upload_dataframe_as_json('all.json',result)

配置数据的共享信息

In [ ]:
online.share_data(data_id,True)

将数据转换为Layer，便于生成地图

In [ ]:
layer=online.prepare_geojson_layer(data_id,'layer')

在Online上根据Layer创建地图，默认底图为天地图

In [ ]:
map_id=online.create_map(layers=[layer],epsgCode=3857,map_title='overlay')

搜索名称为share的群组，配置地图的共享权限信息

In [ ]:
online.share_map(map_id,True)

In [ ]:
online.get_map(map_id=map_id)